### Importações e dicionários

In [1]:
import pandas as pd
import numpy as np
import os, sys
import datetime as dt
from glob import glob
import dask.dataframe as dd
pd.set_option('display.float_format', '{:,.2f}'.format)

In [2]:
# A lógica dessa divisão está em E:\10 Modelo Análise preditiva USP\Preparação Subitens
# Basicamente tentando agrupar por elemento das despesas pouco significativas, 
# ignorar aquelas classificações que quase não são utilizadas e separar aquelas que realmente interessam

agrupar = ['01','03','05','13','14','16','21','31','32','33','34','35','40','43','46','47','48','61','62','65','70',
           '71','73','76','83','91','92','93','94']
ignorar = ['04','08','07','09','10','18','19','20','22','23','24','25','27','37','38','41','42','45','49','59', '62', '64', 
           '65', '66','67','72','74','75', '76','77', '81','82','83','84','86','95','96','97','98','99'] #modalidade 50 fica fora do modelo
dividir = ['11','30','36','39','51','52']
ti = ['3908', '3911', '3957', '3994', '3995', '3997']

substituir = {'0100':'0101','0300':'0301', '1100':'1101', '1187':'1101', '3000':'3099', '3600':'3699', '3900':'3999', '5100':'5199', '5200':'5299'}

### Ler arquivos

In [15]:
arquivos = glob('E:/5 Base PTM/base_mensal/*.parquet')

In [17]:
#base10a21 = pd.read_csv(r'E:\10 Modelo Análise preditiva USP\Compartilhamento USP\dadosprontos2010a2021v2 12.08.2022.csv')
df = pd.DataFrame()
for arquivo in arquivos:
    df = pd.concat([df, pd.read_parquet(arquivo)])

### Excluir Terceiro Setor e Elementos pouco representativos

In [22]:
df['ano_exercicio'] = df.ano_exercicio.astype(int)

In [23]:
df[['cod_subitem', 'ds_subitem']] = df.ds_elemento.str.split(pat=' - ', n=1, expand=True)
df['ds_subitem'] = df['ds_subitem'].str.strip()

In [25]:
df['grupo'] = df.cod_subitem.str[:2]
df['modalidade'] = df.cod_subitem.str[2:4]
df['elemento'] = df.cod_subitem.str[4:6]
df['subitem'] = df.cod_subitem.str[6:8]


In [26]:
print(f'Quantidade de linhas total: {len(df.index)}')
print(f'Quantidade de linhas a serem excluídas: {len(df.loc[df.elemento.isin(ignorar)])}')
print(f'Quantidade de linhas esperada após exclusão: {len(df.index)-len(df.loc[df.elemento.isin(ignorar)])}')
df = df.loc[~df.elemento.isin(ignorar)]
print(f'Quantidade de linhas após a exclusão: {len(df.index)}')
print(f'Quantidade de linhas terceiro setor: {len(df.loc[(df.modalidade=="50")|(df.cod_subitem.str[4:8]=="3975")].index)}')
print(f'Quantidade de linhas esperada após exclusão: {len(df.index)-len(df.loc[(df.modalidade=="50")|(df.cod_subitem.str[4:8]=="3975")].index)}')
df = df.loc[(df.modalidade!='50')&(df.cod_subitem.str[4:8]!='3975')&(df.cod_subitem.str[4:8]!='8500')]
print(f'Quantidade de linhas após exclusão do terceiro setor: {len(df.index)}')

Quantidade de linhas total: 10566027
Quantidade de linhas a serem excluídas: 240538
Quantidade de linhas esperada após exclusão: 10325489
Quantidade de linhas após a exclusão: 10325489
Quantidade de linhas terceiro setor: 104418
Quantidade de linhas esperada após exclusão: 10221071
Quantidade de linhas após exclusão do terceiro setor: 10221071


In [30]:
df

,ano_exercicio,mes_referencia,ds_municipio,ds_orgao,ds_elemento,vl_despesa,cod_subitem,ds_subitem,grupo,modalidade,elemento,subitem,id_despesa
0,2010,1,Adamantina,CENTRO UNIVERSITÁRIO DE ADAMANTINA,31900187 - COMPLEMENTAÇÃO DE APOSENTADORIAS,"15,550.64",31900187,COMPLEMENTAÇÃO DE APOSENTADORIAS,31,90,01,87,NaN
1,2010,1,Adamantina,CENTRO UNIVERSITÁRIO DE ADAMANTINA,31900386 - COMPLEMENTAÇÃO DE PENSÕES - PESSOAL...,"1,428.79",31900386,COMPLEMENTAÇÃO DE PENSÕES - PESSOAL CIVIL,31,90,03,86,NaN
4,2010,1,Adamantina,CENTRO UNIVERSITÁRIO DE ADAMANTINA,31901101 - VENCIMENTOS E SALÁRIOS,"737,578.27",31901101,VENCIMENTOS E SALÁRIOS,31,90,11,01,NaN
5,2010,1,Adamantina,CENTRO UNIVERSITÁRIO DE ADAMANTINA,31901145 - FÉRIAS - ABONO CONSTITUCIONAL,"14,048.54",31901145,FÉRIAS - ABONO CONSTITUCIONAL,31,90,11,45,NaN
6,2010,1,Adamantina,CENTRO UNIVERSITÁRIO DE ADAMANTINA,31901301 - FGTS,"60,599.87",31901301,FGTS,31,90,13,01,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
899713,2022,12,Óleo,PREFEITURA MUNICIPAL DE ÓLEO,33904712 - CONTRIBUIÇÃO PARA O PIS/PASEP,"28,779.67",33904712,CONTRIBUIÇÃO PARA O PIS/PASEP,33,90,47,12,NaN
899714,2022,12,Óleo,PREFEITURA MUNICIPAL DE ÓLEO,33909115 - SENTENÇAS JUDICIAIS DE PEQUENO VALOR,"8,828.52",33909115,SENTENÇAS JUDICIAIS DE PEQUENO VALOR,33,90,91,15,NaN
899715,2022,12,Óleo,PREFEITURA MUNICIPAL DE ÓLEO,44905199 - OUTRAS OBRAS E INSTALAÇÕES,"52,520.16",44905199,OUTRAS OBRAS E INSTALAÇÕES,44,90,51,99,NaN
899716,2022,12,Óleo,PREFEITURA MUNICIPAL DE ÓLEO,44905248 - VEÍCULOS DIVERSOS,"90,900.00",44905248,VEÍCULOS DIVERSOS,44,90,52,48,NaN


### Agrupar elementos e ajustar elemento 40 (mudança de TI em 2020) e subitens 00 por 99

In [33]:
df.reset_index(inplace=True)

In [34]:
df.loc[df.cod_subitem.str[4:8].isin(ti), 'id_despesa'] = '4000'

In [35]:
df.loc[df.elemento.isin(agrupar), 'id_despesa'] = df.elemento+'00'

In [36]:
df.loc[df.elemento.isin(dividir), 'id_despesa'] = df.elemento+df.subitem

In [37]:
df.id_despesa.replace(substituir, inplace=True)

In [38]:
df.loc[(df.id_despesa=='0100')].head()

,index,ano_exercicio,mes_referencia,ds_municipio,ds_orgao,ds_elemento,vl_despesa,cod_subitem,ds_subitem,grupo,modalidade,elemento,subitem,id_despesa


In [39]:
df.loc[df.id_despesa.isna()]

,index,ano_exercicio,mes_referencia,ds_municipio,ds_orgao,ds_elemento,vl_despesa,cod_subitem,ds_subitem,grupo,modalidade,elemento,subitem,id_despesa


In [40]:
len(df.index)

10221071

In [41]:
df.loc[df.ano_exercicio.isna()]

,index,ano_exercicio,mes_referencia,ds_municipio,ds_orgao,ds_elemento,vl_despesa,cod_subitem,ds_subitem,grupo,modalidade,elemento,subitem,id_despesa


### Incluir Entidade ID

In [42]:
entidade = pd.read_excel('de.para.entidades.xlsx')

In [43]:
entidade['ds_municipio'] = entidade.ds_municipio.str.strip()
entidade['ds_orgao'] = entidade.ds_orgao.str.strip()

In [44]:
df['ds_municipio'] = df.ds_municipio.str.strip()
df['ds_orgao'] = df.ds_orgao.str.strip()
df.ds_orgao = df.ds_orgao.str.replace(' +', ' ', regex=True)

In [45]:
len(df.index)

10221071

In [46]:
len(df.merge(entidade, how='left'))

10221071

In [47]:
df = df.merge(entidade, how='left')

### Gerar a base

In [49]:
df.groupby(['ano_exercicio', 'mes_referencia','cd_municipio_ibge', 'entidade_id', 
            'id_despesa', 'ds_municipio', 'ds_orgao', 'ds_tp_entidade']).sum().reset_index().to_parquet('E:/fgv/base2022subitensagrupadospormes.parquet', index=False)

C:\Users\fmacieira\AppData\Local\Temp\ipykernel_9312\676227270.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby(['ano_exercicio', 'mes_referencia','cd_municipio_ibge', 'entidade_id', 'id_despesa', 'ds_municipio', 'ds_orgao', 'ds_tp_entidade']).sum().reset_index().to_parquet('E:/fgv/base2022subitensagrupadospormes.parquet', index=False)


### Lixo

In [98]:
df.loc[(df.entidade_id == 1)&(df.id_despesa == '3300')]

,ano_exercicio,ds_municipio,ds_orgao,ds_elemento,vl_despesa,cod_subitem,ds_subitem,grupo,modalidade,elemento,subitem,id_despesa,entidade_id,cd_municipio_ibge,ds_tp_entidade


In [99]:
df.loc[(df.id_despesa == '3300')]

,ano_exercicio,ds_municipio,ds_orgao,ds_elemento,vl_despesa,cod_subitem,ds_subitem,grupo,modalidade,elemento,subitem,id_despesa,entidade_id,cd_municipio_ibge,ds_tp_entidade
487,2022,Aguaí,CÂMARA MUNICIPAL DE AGUAÍ,33903308 - PEDÁGIOS,132.90,33903308,PEDÁGIOS,33,90,33,08,3300,10446,3500303,CÂMARA MUNICIPAL
488,2022,Aguaí,CÂMARA MUNICIPAL DE AGUAÍ,33903399 - OUTRAS DESPESAS COM LOCOMOÇÃO,572.86,33903399,OUTRAS DESPESAS COM LOCOMOÇÃO,33,90,33,99,3300,10446,3500303,CÂMARA MUNICIPAL
558,2022,Aguaí,PREFEITURA MUNICIPAL DE AGUAÍ,33903301 - PASSAGENS PARA O PAÍS,"19,187.63",33903301,PASSAGENS PARA O PAÍS,33,90,33,01,3300,10445,3500303,PREFEITURA MUNICIPAL
559,2022,Aguaí,PREFEITURA MUNICIPAL DE AGUAÍ,33903399 - OUTRAS DESPESAS COM LOCOMOÇÃO,"28,193.89",33903399,OUTRAS DESPESAS COM LOCOMOÇÃO,33,90,33,99,3300,10445,3500303,PREFEITURA MUNICIPAL
665,2022,Agudos,CÂMARA MUNICIPAL DE AGUDOS,33903308 - PEDÁGIOS,"10,246.29",33903308,PEDÁGIOS,33,90,33,08,3300,2256,3500709,CÂMARA MUNICIPAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121091,2022,Várzea Paulista,CÂMARA MUNICIPAL DE VÁRZEA PAULISTA,33903399 - OUTRAS DESPESAS COM LOCOMOÇÃO,48.00,33903399,OUTRAS DESPESAS COM LOCOMOÇÃO,33,90,33,99,3300,15359,3556503,CÂMARA MUNICIPAL
122123,2022,Águas de São Pedro,PREFEITURA MUNICIPAL DE ÁGUAS DE SÃO PEDRO,33903399 - OUTRAS DESPESAS COM LOCOMOÇÃO,"86,988.25",33903399,OUTRAS DESPESAS COM LOCOMOÇÃO,33,90,33,99,3300,24073,3500600,PREFEITURA MUNICIPAL
122487,2022,Álvaro de Carvalho,CÂMARA MUNICIPAL DE ÁLVARO DE CARVALHO,33903301 - PASSAGENS PARA O PAÍS,"1,585.89",33903301,PASSAGENS PARA O PAÍS,33,90,33,01,3300,32297,3501400,CÂMARA MUNICIPAL
122488,2022,Álvaro de Carvalho,CÂMARA MUNICIPAL DE ÁLVARO DE CARVALHO,33903399 - OUTRAS DESPESAS COM LOCOMOÇÃO,"5,942.94",33903399,OUTRAS DESPESAS COM LOCOMOÇÃO,33,90,33,99,3300,32297,3501400,CÂMARA MUNICIPAL


In [93]:
df.head()

,ano_exercicio,ds_municipio,ds_orgao,ds_elemento,vl_despesa,cod_subitem,ds_subitem,grupo,modalidade,elemento,subitem,id_despesa,entidade_id,cd_municipio_ibge,ds_tp_entidade
0,2022,Adamantina,CENTRO UNIVERSITÁRIO DE ADAMANTINA,31900199 - OUTRAS APOSENTADORIAS,"1,234,774.07",31900199,OUTRAS APOSENTADORIAS,31,90,01,99,0101,16801,3500105,AUTARQUIA
1,2022,Adamantina,CENTRO UNIVERSITÁRIO DE ADAMANTINA,31900399 - OUTRAS PENSÕES,"23,820.15",31900399,OUTRAS PENSÕES,31,90,03,99,0301,16801,3500105,AUTARQUIA
2,2022,Adamantina,CENTRO UNIVERSITÁRIO DE ADAMANTINA,31901101 - VENCIMENTOS E SALÁRIOS,"20,206,523.98",31901101,VENCIMENTOS E SALÁRIOS,31,90,11,01,1101,16801,3500105,AUTARQUIA
3,2022,Adamantina,CENTRO UNIVERSITÁRIO DE ADAMANTINA,31901104 - ADICIONAL NOTURNO,"378,339.98",31901104,ADICIONAL NOTURNO,31,90,11,04,1104,16801,3500105,AUTARQUIA
4,2022,Adamantina,CENTRO UNIVERSITÁRIO DE ADAMANTINA,31901105 - INCORPORAÇÕES,"52,632.61",31901105,INCORPORAÇÕES,31,90,11,05,1105,16801,3500105,AUTARQUIA


In [10]:
base10a21.head()

,ano_exercicio,cd_municipio_ibge,entidade_id,ds_municipio,ds_orgao,ds_tp_entidade,tipo_orgao,id_despesa,vl_despesa,RCL,pib_nominal,população,atividade_principal,quantidade_total_vagas,vereadores,area,alunos_escola
0,2010,3501103,1,Alto Alegre,PREFEITURA MUNICIPAL DE ALTO ALEGRE,PREFEITURA MUNICIPAL,prefeitura,1101,"3,208,861.90","10,434,035.00",110457,4110,"Agricultura, inclusive apoio à agricultura e a...",468,9,318,297
1,2010,3501103,1,Alto Alegre,PREFEITURA MUNICIPAL DE ALTO ALEGRE,PREFEITURA MUNICIPAL,prefeitura,1143,"247,849.28","10,434,035.00",110457,4110,"Agricultura, inclusive apoio à agricultura e a...",468,9,318,297
2,2010,3501103,1,Alto Alegre,PREFEITURA MUNICIPAL DE ALTO ALEGRE,PREFEITURA MUNICIPAL,prefeitura,1160,"143,573.18","10,434,035.00",110457,4110,"Agricultura, inclusive apoio à agricultura e a...",468,9,318,297
3,2010,3501103,1,Alto Alegre,PREFEITURA MUNICIPAL DE ALTO ALEGRE,PREFEITURA MUNICIPAL,prefeitura,1300,"1,016,457.25","10,434,035.00",110457,4110,"Agricultura, inclusive apoio à agricultura e a...",468,9,318,297
4,2010,3501103,1,Alto Alegre,PREFEITURA MUNICIPAL DE ALTO ALEGRE,PREFEITURA MUNICIPAL,prefeitura,3001,"461,233.63","10,434,035.00",110457,4110,"Agricultura, inclusive apoio à agricultura e a...",468,9,318,297


In [187]:
pd.pivot_table(df.loc[df.entidade_id.isna()], index=['ds_municipio', 'ds_orgao'], columns='ano_exercicio', values='vl_despesa', aggfunc='sum')

ano_exercicio                                                              2010  \
ds_municipio    ds_orgao                                                          
Barretos        FUNDAÇÃO EDUCACIONAL DE BARRETOS                  30,678,270.00   
Cordeirópolis   HOSPITAL E MATERNIDADE DE CORDEIRÓPOLIS            5,376,924.70   
Ilha Solteira   FUNEDISA                                             419,341.14   
Itapira         EMPRESA MUNICIPAL DE URBANISMO E HAB. DE ITAPIRA     745,738.89   
Limeira         CODEL - COMPANHIA DE DESENVOLVIMENTO DE LIMEIRA       76,923.12   
                SERVIÇO AUTÔNOMO DE ÁGUA E ESGOTO DE LIMEIRA       6,856,607.89   
Martinópolis    FUNDAÇÃO DE ENSINO LINCONL DE ANDRADE JUNQUEIRA       72,296.40   
Matão           CAEMA - COMPANHIA DE ÁGUA E ESGOTO DE MATÃO       15,329,107.46   
Monte Aprazível FUNDAÇÃO EDUCACIONAL MONTE APRAZÍVEL                  45,592.62   
Pompéia         ADMINISTRAÇÃO INDIRETA - I.A.S.F.E.M.P.              789,047.52   
Porto Ferreira  SAEF-SERV.DE ÁGUA E ESGOTO DE PORTO FERREIRA      14,843,867.33   
Serra Negra     EMPRESA MUNICIPAL DE HABITAÇÃO DE SERRA NEGRA             95.60   
Sorocaba        NÚCLEO DE PLANEJAMENTO REGIONAL S-A - NUPLAN                NaN   
Sumaré          DAE                                               37,565,546.04   
São Carlos      FUNDO MUNICIPAL DE SAÚDE-ADMINISTRAÇÃO DIRETA    125,617,186.18   
Urânia          FUNDAÇÃO DE SERV.ASSIST. E COMUM. DO MUN. URÂNIA      75,074.88   
Votorantim      SERVIÇO AUTONOMO DE ÁGUA E ESGOTO                 20,850,709.36   

ano_exercicio                                                              2011  \
ds_municipio    ds_orgao                                                          
Barretos        FUNDAÇÃO EDUCACIONAL DE BARRETOS                  34,119,982.12   
Cordeirópolis   HOSPITAL E MATERNIDADE DE CORDEIRÓPOLIS            5,840,894.24   
Ilha Solteira   FUNEDISA                                             481,067.68   
Itapira         EMPRESA MUNICIPAL DE URBANISMO E HAB. DE ITAPIRA            NaN   
Limeira         CODEL - COMPANHIA DE DESENVOLVIMENTO DE LIMEIRA      605,477.10   
                SERVIÇO AUTÔNOMO DE ÁGUA E ESGOTO DE LIMEIRA       7,500,828.29   
Martinópolis    FUNDAÇÃO DE ENSINO LINCONL DE ANDRADE JUNQUEIRA             NaN   
Matão           CAEMA - COMPANHIA DE ÁGUA E ESGOTO DE MATÃO       16,854,586.10   
Monte Aprazível FUNDAÇÃO EDUCACIONAL MONTE APRAZÍVEL                  52,223.78   
Pompéia         ADMINISTRAÇÃO INDIRETA - I.A.S.F.E.M.P.                     NaN   
Porto Ferreira  SAEF-SERV.DE ÁGUA E ESGOTO DE PORTO FERREIRA      10,485,691.39   
Serra Negra     EMPRESA MUNICIPAL DE HABITAÇÃO DE SERRA NEGRA            234.51   
Sorocaba        NÚCLEO DE PLANEJAMENTO REGIONAL S-A - NUPLAN                NaN   
Sumaré          DAE                                               43,491,640.47   
São Carlos      FUNDO MUNICIPAL DE SAÚDE-ADMINISTRAÇÃO DIRETA    130,932,758.21   
Urânia          FUNDAÇÃO DE SERV.ASSIST. E COMUM. DO MUN. URÂNIA      70,956.35   
Votorantim      SERVIÇO AUTONOMO DE ÁGUA E ESGOTO                 23,566,235.60   

ano_exercicio                                                              2012  \
ds_municipio    ds_orgao                                                          
Barretos        FUNDAÇÃO EDUCACIONAL DE BARRETOS                  37,101,223.56   
Cordeirópolis   HOSPITAL E MATERNIDADE DE CORDEIRÓPOLIS            6,966,377.79   
Ilha Solteira   FUNEDISA                                             513,206.09   
Itapira         EMPRESA MUNICIPAL DE URBANISMO E HAB. DE ITAPIRA            NaN   
Limeira         CODEL - COMPANHIA DE DESENVOLVIMENTO DE LIMEIRA    1,161,341.98   
                SERVIÇO AUTÔNOMO DE ÁGUA E ESGOTO DE LIMEIRA       7,812,110.85   
Martinópolis    FUNDAÇÃO DE ENSINO LINCONL DE ANDRADE JUNQUEIRA             NaN   
Matão           CAEMA - COMPANHIA DE ÁGUA E ESGOTO DE MATÃO       16,786,354.38   
Monte Aprazíve

In [191]:
agrupado = df.groupby(['ano_exercicio','cd_municipio_ibge', 'entidade_id', 'id_despesa']).sum().reset_index()

In [192]:
agrupado.head()

,ano_exercicio,cd_municipio_ibge,entidade_id,id_despesa,vl_despesa
0,2010,"3,500,105.00","16,799.00",0100,"2,782,080.93"
1,2010,"3,500,105.00","16,799.00",0300,"1,059,297.99"
2,2010,"3,500,105.00","16,799.00",0400,"273,400.19"
3,2010,"3,500,105.00","16,799.00",1101,"8,897,560.13"
4,2010,"3,500,105.00","16,799.00",1137,388.20


In [201]:
agrupado.loc[(agrupado.vl_despesa<10000)]

,ano_exercicio,cd_municipio_ibge,entidade_id,id_despesa,vl_despesa
4,2010,"3,500,105.00","16,799.00",1137,388.20
13,2010,"3,500,105.00","16,799.00",3004,198.00
15,2010,"3,500,105.00","16,799.00",3009,635.48
17,2010,"3,500,105.00","16,799.00",3011,"8,021.00"
19,2010,"3,500,105.00","16,799.00",3015,700.00
...,...,...,...,...,...
1259060,2021,"3,557,303.00","13,366.00",5180,"5,582.75"
1259062,2021,"3,557,303.00","13,366.00",5206,"1,552.11"
1259063,2021,"3,557,303.00","13,366.00",5233,696.00
1259064,2021,"3,557,303.00","13,366.00",5235,"6,650.00"


In [202]:
df.loc[(df.entidade_id==13366)&(df.ano_exercicio==2021)&(df.id_despesa=='5238')]

,ano_exercicio,ds_municipio,ds_orgao,ds_elemento,vl_despesa,cod_subitem,ds_subitem,grupo,modalidade,elemento,subitem,id_despesa,entidade_id,cd_municipio_ibge,ds_tp_entidade
1285952,2021,Estiva Gerbi,CÂMARA MUNICIPAL DE ESTIVA GERBI,"44905238 - MÁQUINAS, FERRAMENTAS E UTENSÍLIOS ...","1,050.00",44905238,"MÁQUINAS, FERRAMENTAS E UTENSÍLIOS DE OFICINA",44,90,52,38,5238,"13,366.00","3,557,303.00",CÂMARA MUNICIPAL


In [203]:
agrupado.loc[(agrupado.vl_despesa>10000)&(agrupado.id_despesa=='5238')]

,ano_exercicio,cd_municipio_ibge,entidade_id,id_despesa,vl_despesa
7153,2010,"3,504,107.00","59,680.00",5238,"18,392.00"
9406,2010,"3,505,708.00","4,439.00",5238,"12,042.81"
10145,2010,"3,506,003.00",170.00,5238,"77,248.72"
15828,2010,"3,509,403.00","19,639.00",5238,"10,216.00"
18955,2010,"3,511,003.00","11,065.00",5238,"15,744.19"
...,...,...,...,...,...
1255628,2021,"3,555,901.00","14,911.00",5238,"34,845.00"
1255771,2021,"3,556,008.00","18,423.00",5238,"18,755.30"
1256240,2021,"3,556,206.00","17,946.00",5238,"16,495.71"
1257393,2021,"3,556,701.00","7,944.00",5238,"33,920.46"


In [207]:
agrupado.groupby('id_despesa')['vl_despesa'].agg(['min', 'mean', 'max', 'count']).to_excel('E:/grupos de elementos.xlsx')

In [213]:
pd.pivot_table(df, index=['ds_municipio', 'ds_orgao', 'id_despesa'], columns='ano_exercicio', values='vl_despesa', aggfunc='sum').reset_index().to_excel('E:/teste soma elementos.xlsx', index=False)

In [214]:
agrupado.loc[agrupado.ano_exercicio==2021].groupby('id_despesa')['vl_despesa'].agg(['min', 'mean', 'max', 'count']).sort_values('count')

,min,mean,max,count
id_despesa,,,,
3913,"30,091.85","30,091.85","30,091.85",1
4200,"5,349.00","5,349.00","5,349.00",1
5219,"8,896.00","8,896.00","8,896.00",1
1160,"24,700.00","24,700.00","24,700.00",1
3637,"4,000.00","4,000.00","4,000.00",1
...,...,...,...,...
3016,100.00,"134,036.55","13,743,513.96",1585
3943,680.75,"1,816,945.13","88,071,561.62",1587
1101,"5,159.33","19,692,841.75","1,418,703,343.19",1653


In [215]:
agrupado.loc[agrupado.ano_exercicio==2021].groupby('id_despesa')['vl_despesa'].agg(['min', 'mean', 'max', 'count']).reset_index().to_excel('E:/Elementosem2021.xlsx', index = False)

In [216]:
df.columns

Index(['ano_exercicio', 'ds_municipio', 'ds_orgao', 'ds_elemento',
       'vl_despesa', 'cod_subitem', 'ds_subitem', 'grupo', 'modalidade',
       'elemento', 'subitem', 'id_despesa', 'entidade_id', 'cd_municipio_ibge',
       'ds_tp_entidade'],
      dtype='object')

In [296]:
agrupado = df.groupby(['ano_exercicio','cd_municipio_ibge', 'entidade_id', 'ds_tp_entidade', 'id_despesa']).sum().reset_index()
agrupado.groupby(['id_despesa','ds_tp_entidade'])['vl_despesa'].agg(['min', 'mean', 'max', 'count']).reset_index().to_excel('E:/grupamento por id e tp entidade.xlsx', index=False)

In [298]:
agrupado.groupby(['id_despesa','ds_tp_entidade'])['vl_despesa'].agg(['min', 'mean', 'max', 'count']).reset_index().to_excel('E:/grupamento por id e tp entidade.xlsx', index=False)

In [225]:
elementos = pd.read_excel('E:/Elementos.xlsx', dtype={'id_despesa':'object', 'NOME DO CÓDIGO':'object'})[['id_despesa', 'NOME DO CÓDIGO']].drop_duplicates(subset='id_despesa')

In [228]:
elementos.loc[elementos.duplicated(subset='id_despesa')]

,id_despesa,NOME DO CÓDIGO


In [229]:
elementos.to_excel('E:/Elementos.xlsx', index=False)

In [323]:
df.loc[df.id_despesa=='3911']

,ano_exercicio,ds_municipio,ds_orgao,ds_elemento,vl_despesa,cod_subitem,ds_subitem,grupo,modalidade,elemento,subitem,id_despesa,entidade_id,cd_municipio_ibge,ds_tp_entidade
62,2010,Adamantina,CENTRO UNIVERSITÁRIO DE ADAMANTINA,33903911 - LOCAÇÃO DE SOFTWARES,"29,615.56",33903911,LOCAÇÃO DE SOFTWARES,33,90,39,11,3911,"16,801.00","3,500,105.00",AUTARQUIA
139,2010,Adamantina,CÂMARA MUNICIPAL DE ADAMANTINA,33903911 - LOCAÇÃO DE SOFTWARES,"17,901.70",33903911,LOCAÇÃO DE SOFTWARES,33,90,39,11,3911,"16,800.00","3,500,105.00",CÂMARA MUNICIPAL
219,2010,Adamantina,PREFEITURA MUNICIPAL DE ADAMANTINA,33903911 - LOCAÇÃO DE SOFTWARES,"232,284.89",33903911,LOCAÇÃO DE SOFTWARES,33,90,39,11,3911,"16,799.00","3,500,105.00",PREFEITURA MUNICIPAL
290,2010,Adolfo,CÂMARA MUNICIPAL DE ADOLFO,33903911 - LOCAÇÃO DE SOFTWARES,"6,928.40",33903911,LOCAÇÃO DE SOFTWARES,33,90,39,11,3911,"2,249.00","3,500,204.00",CÂMARA MUNICIPAL
344,2010,Adolfo,PREFEITURA MUNICIPAL DE ADOLFO,33903911 - LOCAÇÃO DE SOFTWARES,"36,925.47",33903911,LOCAÇÃO DE SOFTWARES,33,90,39,11,3911,"2,248.00","3,500,204.00",PREFEITURA MUNICIPAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345390,2021,Rincão,PREFEITURA MUNICIPAL DE RINCÃO,33903911 - LOCAÇÃO DE SOFTWARES,"111,699.04",33903911,LOCAÇÃO DE SOFTWARES,33,90,39,11,3911,"1,815.00","3,543,709.00",PREFEITURA MUNICIPAL
1357448,2021,Sumaré,CÂMARA MUNICIPAL DE SUMARÉ,33903911 - LOCAÇÃO DE SOFTWARES,"20,787.21",33903911,LOCAÇÃO DE SOFTWARES,33,90,39,11,3911,"10,218.00","3,552,403.00",CÂMARA MUNICIPAL
1359633,2021,São Carlos,SAAE - SERVIÇO AUTÔNOMO DE ÁGUA E ESGOTO,33903911 - LOCAÇÃO DE SOFTWARES,"13,247.00",33903911,LOCAÇÃO DE SOFTWARES,33,90,39,11,3911,"10,022.00","3,548,906.00",AUTARQUIA
1360544,2021,São José do Rio Pardo,INSTITUTO MUNICIPAL DE PREVIDENCIA,33903911 - LOCAÇÃO DE SOFTWARES,"22,333.32",33903911,LOCAÇÃO DE SOFTWARES,33,90,39,11,3911,"16,357.00","3,549,706.00",ENTIDADE DE PREVIDÊNCIA


In [320]:
df.loc[(df.elemento=='39')&(df.subitem=='08')]['ds_elemento'].unique()

array(['33903908 - MANUTENÇÃO DE SOFTWARE',
       '33713908 - MANUTENÇÃO DE SOFTWARE - TRANSFERÊNCIAS A CONSÓRCIOS PÚBLICOS',
       '33913908 - MANUTENÇÃO DE SOFTWARE - INTRA-ORÇAMENTÁRIO'],
      dtype=object)

In [244]:
df.head()

,ano_exercicio,ds_municipio,ds_orgao,ds_elemento,vl_despesa,cod_subitem,ds_subitem,grupo,modalidade,elemento,subitem,id_despesa
0,2010,Adamantina,CENTRO UNIVERSITÁRIO DE ADAMANTINA,31900187 - COMPLEMENTAÇÃO DE APOSENTADORIAS,"174,090.99",31900187,COMPLEMENTAÇÃO DE APOSENTADORIAS,31,90,01,87,0100
1,2010,Adamantina,CENTRO UNIVERSITÁRIO DE ADAMANTINA,31900199 - OUTRAS APOSENTADORIAS,"16,643.38",31900199,OUTRAS APOSENTADORIAS,31,90,01,99,0100
2,2010,Adamantina,CENTRO UNIVERSITÁRIO DE ADAMANTINA,31900386 - COMPLEMENTAÇÃO DE PENSÕES - PESSOAL...,"14,005.31",31900386,COMPLEMENTAÇÃO DE PENSÕES - PESSOAL CIVIL,31,90,03,86,0300
3,2010,Adamantina,CENTRO UNIVERSITÁRIO DE ADAMANTINA,31900399 - OUTRAS PENSÕES,"3,018.11",31900399,OUTRAS PENSÕES,31,90,03,99,0300
4,2010,Adamantina,CENTRO UNIVERSITÁRIO DE ADAMANTINA,31900401 - PROFESSORES SUBSTITUTOS / VISITANTES,"1,049.28",31900401,PROFESSORES SUBSTITUTOS / VISITANTES,31,90,04,01,0400


In [398]:
teste = pd.read_csv('E:/ptm2010a2021.csv')

In [399]:
teste.head()

,ano_exercicio,ds_municipio,ds_orgao,ds_elemento,vl_despesa
0,"2,010.00",Adamantina,CENTRO UNIVERSITÁRIO DE ADAMANTINA,31900187 - COMPLEMENTAÇÃO DE APOSENTADORIAS,"174,090.99"
1,"2,010.00",Adamantina,CENTRO UNIVERSITÁRIO DE ADAMANTINA,31900199 - OUTRAS APOSENTADORIAS,"16,643.38"
2,"2,010.00",Adamantina,CENTRO UNIVERSITÁRIO DE ADAMANTINA,31900386 - COMPLEMENTAÇÃO DE PENSÕES - PESSOAL...,"14,005.31"
3,"2,010.00",Adamantina,CENTRO UNIVERSITÁRIO DE ADAMANTINA,31900399 - OUTRAS PENSÕES,"3,018.11"
4,"2,010.00",Adamantina,CENTRO UNIVERSITÁRIO DE ADAMANTINA,31900401 - PROFESSORES SUBSTITUTOS / VISITANTES,"1,049.28"


In [401]:
teste.loc[teste.ds_elemento.str[4:8]=='1143']

,ano_exercicio,ds_municipio,ds_orgao,ds_elemento,vl_despesa
120,"2,010.00",Adamantina,CÂMARA MUNICIPAL DE ADAMANTINA,31901143 - 13º SALÁRIO,"19,745.13"
167,"2,010.00",Adamantina,PREFEITURA MUNICIPAL DE ADAMANTINA,31901143 - 13º SALÁRIO,"1,059,666.71"
275,"2,010.00",Adolfo,CÂMARA MUNICIPAL DE ADOLFO,31901143 - 13º SALÁRIO,"8,950.18"
309,"2,010.00",Adolfo,PREFEITURA MUNICIPAL DE ADOLFO,31901143 - 13º SALÁRIO,"272,633.16"
386,"2,010.00",Aguaí,CÂMARA MUNICIPAL DE AGUAÍ,31901143 - 13º SALÁRIO,"18,415.46"
...,...,...,...,...,...
1389674,"2,021.00",Álvares Machado,CÂMARA MUNICIPAL DE ÁLVARES MACHADO,31901143 - 13º SALÁRIO,"48,734.54"
1389696,"2,021.00",Álvares Machado,PREFEITURA MUNICIPAL DE ÁLVARES MACHADO,31901143 - 13º SALÁRIO,"1,874,333.55"
1389746,"2,021.00",Álvaro de Carvalho,CÂMARA MUNICIPAL DE ÁLVARO DE CARVALHO,31901143 - 13º SALÁRIO,"8,623.14"
1389791,"2,021.00",Álvaro de Carvalho,PREFEITURA MUNICIPAL DE ÁLVARO DE CARVALHO,31901143 - 13º SALÁRIO,"335,622.42"


In [18]:
df.loc[(df.modalidade=='50')|(df.cod_subitem.str[4:8]=='3975')|(df.cod_subitem.str[4:8]=='8500')]

,ano_exercicio,ds_municipio,ds_orgao,ds_elemento,vl_despesa,cod_subitem,ds_subitem,grupo,modalidade,elemento,subitem
32,2022,Adamantina,CENTRO UNIVERSITÁRIO DE ADAMANTINA,33503903 - ACORDO DE COOPERAÇÃO,"20,599.61",33503903,ACORDO DE COOPERAÇÃO,33,50,39,03
33,2022,Adamantina,CENTRO UNIVERSITÁRIO DE ADAMANTINA,33503906 - CONVÊNIO,"58,816.48",33503906,CONVÊNIO,33,50,39,06
34,2022,Adamantina,CENTRO UNIVERSITÁRIO DE ADAMANTINA,33504300 - SUBVENÇÕES SOCIAIS,"10,362,452.46",33504300,SUBVENÇÕES SOCIAIS,33,50,43,00
237,2022,Adamantina,PREFEITURA MUNICIPAL DE ADAMANTINA,33503903 - ACORDO DE COOPERAÇÃO,"6,600.00",33503903,ACORDO DE COOPERAÇÃO,33,50,39,03
238,2022,Adamantina,PREFEITURA MUNICIPAL DE ADAMANTINA,33503951 - SERVIÇOS MÉDICO-HOSPITALARES PRESTA...,"22,654,640.05",33503951,SERVIÇOS MÉDICO-HOSPITALARES PRESTADOS EM UNID...,33,50,39,51
...,...,...,...,...,...,...,...,...,...,...,...
126250,2022,Águas de Santa Bárbara,PREFEITURA MUNICIPAL DE ÁGUAS DE SANTA BÁRBARA,33504300 - SUBVENÇÕES SOCIAIS,"540,000.00",33504300,SUBVENÇÕES SOCIAIS,33,50,43,00
126535,2022,Álvares Florence,PREFEITURA MUNICIPAL DE ÁLVARES FLORENCE,33504300 - SUBVENÇÕES SOCIAIS,"396,332.89",33504300,SUBVENÇÕES SOCIAIS,33,50,43,00
126692,2022,Álvares Machado,PREFEITURA MUNICIPAL DE ÁLVARES MACHADO,33504300 - SUBVENÇÕES SOCIAIS,"5,988,274.30",33504300,SUBVENÇÕES SOCIAIS,33,50,43,00
126823,2022,Álvaro de Carvalho,PREFEITURA MUNICIPAL DE ÁLVARO DE CARVALHO,33504300 - SUBVENÇÕES SOCIAIS,"357,619.92",33504300,SUBVENÇÕES SOCIAIS,33,50,43,00


In [52]:
entidade.loc[entidade.cd_municipio_ibge.isna()]

,entidade_id,cd_municipio_ibge,ds_municipio,ds_orgao,ds_tp_entidade


In [53]:
entidade.head()

,entidade_id,cd_municipio_ibge,ds_municipio,ds_orgao,ds_tp_entidade
0,1,3501103,Alto Alegre,PREFEITURA MUNICIPAL DE ALTO ALEGRE,PREFEITURA MUNICIPAL
1,2,3501103,Alto Alegre,CÂMARA MUNICIPAL DE ALTO ALEGRE,CÂMARA MUNICIPAL
2,30,3502002,Analândia,PREFEITURA MUNICIPAL DE ANALÂNDIA,PREFEITURA MUNICIPAL
3,31,3502002,Analândia,CÂMARA MUNICIPAL DE ANALÂNDIA,CÂMARA MUNICIPAL
4,73,3503356,Arco-Íris,PREFEITURA MUNICIPAL DE ARCO-ÍRIS,PREFEITURA MUNICIPAL


In [58]:
df.columns

Index(['ano_exercicio', 'ds_municipio', 'ds_orgao', 'ds_elemento',
       'vl_despesa', 'cod_subitem', 'ds_subitem', 'grupo', 'modalidade',
       'elemento', 'subitem', 'id_despesa', 'entidade_id', 'cd_municipio_ibge',
       'ds_tp_entidade'],
      dtype='object')

In [91]:
df.loc[df.entidade_id.isna(), 'ds_orgao'].unique()

array([], dtype=object)